In [26]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables,MaxMultiply):
#     print 'BranchedVariables',BranchedVariables
    NotBranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            NotBranched.append(i)
    MultiplyVar=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))

    f=0
    while len(MultiplyVar)<MaxMultiply:
        if f not in BranchedVariables:
            MultiplyVar.append(f)
        f=f+1
    MultiplyVar.sort()
    print 'MultiplyVar', MultiplyVar
    NodeNum=n-len(MultiplyVar)
    
    print 'NodeNum', NodeNum
    for i in MultiplyVar:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in MultiplyVar: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in NotBranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in NotBranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in NotBranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     for j in range (50):
#         print 'j',j
#         for i in range (749):    
#         print 'd',j,d[:,j]
#     print len(d[:,1])
#     print len(d[1,:])
#     print 'unbranched',unbranched
#     print c,111111111111111111111111, d
    return c, d,MultiplyVar,NotBranched
#     print c, d,unbranched

In [27]:
def SecondModel (cc,dd, BranchedVariables,MultiplyVar,NotBranched):
#     NotBranched=list()
#     for i in range (n):
#         if i not in (BranchedVariables):
#             NotBranched.append(i)
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    LenMultiplyVar=len(MultiplyVar)
    lenNotbranched=len(NotBranched)
    ConstNum=numrows/LenMultiplyVar
    print 'ConstNum',ConstNum
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))
    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in MultiplyVar: 
                indj=MultiplyVar.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
   
    
    for t in range(LenMultiplyVar):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in MultiplyVar and i > MultiplyVar[t]:
                indx=MultiplyVar.index(i)
                indy=MultiplyVar.index(MultiplyVar[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,MultiplyVar[t]+VarNum])
                    dd[(indx)*ConstNum+j,MultiplyVar[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<LenMultiplyVar-2 :
                    for p in range(LenMultiplyVar-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<LenMultiplyVar-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(LenMultiplyVar-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
   
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [28]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables,MaxMultiply,MultiplyVar): 
    BranchedLen=len(BranchedVariables)
#     unbranched=list()
#     for i in range (n):
#         if i not in (BranchedVariables):
#             unbranched.append(i)
#     f=0
#     while len(unbranched)<MaxMultiply:
#         if f not in BranchedVariables:
#             unbranched.append(f)
#         f=f+1
#     unbranched.sort()
    print 'unbranched2222', MultiplyVar
    LenMultiplyVar=len(MultiplyVar)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LenMultiplyVar*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LenMultiplyVar*((2 * m) + (4 * n)),1)))

    for i in range (LenMultiplyVar):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (2):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
# #     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [29]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
        MaxBranch=18
        MaxMultiply=8         
        self.times_called += 1
        print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
        print 'values',values
        print 'branchedNames',branchedNames
        print 'branched',branched

        if len(values)>0:  
            if len(branched)<=MaxBranch:
                BranchedMatrix.append([branched,values])
    #             BranchedValue.append(values)
    #             print 'BranchedMatrix',BranchedMatrix

                if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
                    SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched,MaxMultiply)
                    SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],branched,SecModelParameterOutput[2],SecModelParameterOutput[3])
#                     print 'SecondModelOutput[1]',SecondModelOutput[1]
#                     print 'SecondModelOutput[0]',SecondModelOutput[0]
                    if SecondModelOutput[1]>0.000001:
        #                 print 'bayad cut bezane'
                        output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0],MaxMultiply,SecModelParameterOutput[2])
    #                     print 'output',output[0]
    #                     print 'output[1]',output[1]
                        lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
    #                     print 'lhs',lhs[0]
                        self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
                        print 'cut added'



In [30]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))
BranchOrdering=[]
for i in range (n):
    BranchOrdering.append((i,i+1,c.order.branch_direction.default))
c.order.set(BranchOrdering)
print 'order',c.order.get()
admipex3(c)
CutCplex(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


order [(0, 1, 0), (1, 2, 0), (2, 3, 0), (3, 4, 0), (4, 5, 0), (5, 6, 0), (6, 7, 0), (7, 8, 0), (8, 9, 0), (9, 10, 0), (10, 11, 0), (11, 12, 0), (12, 13, 0), (13, 14, 0), (14, 15, 0), (15, 16, 0), (16, 17, 0), (17, 18, 0), (18, 19, 0), (19, 20, 0), (20, 21, 0), (21, 22, 0), (22, 23, 0), (23, 24, 0), (24, 25, 0), (25, 26, 0), (26, 27, 0), (27, 28, 0), (28, 29, 0), (29, 30, 0)]
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_MIP_Cuts_Cliques                        -1
CPXPARAM_MIP_Cuts_Covers                         -1
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Cuts_FlowCovers                     -1
CPXPARAM_MIP_Cuts_Implied                        -1
CPXPARAM_MIP_Cuts_GUBCovers                      -1
CPXPARAM_MIP_Cuts_Gomory                         -1
CPXPARAM_MIP_Cuts_PathCut                        -1
CPXPARAM_MIP_Cuts_MIRCut                         -1
CPXPARAM_MIP_Cuts_Disjunctive                    -1
C

Removing perturbation.
self.times_called 14
get_num_nodes 6
values [1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x18', 'x19', 'x22', 'x24', 'x27']
branched [18, 19, 22, 24, 27]
      6     8     1413.8633     7     1171.0000     1422.8501       25   21.51%             x18 U      6      5      5
self.times_called 15
get_num_nodes 7
values [-0.0, 1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x16', 'x18', 'x19', 'x22', 'x24', 'x27']
branched [16, 18, 19, 22, 24, 27]
MultiplyVar [0, 1, 2, 3, 4, 5, 6, 7]
NodeNum 22
ConstNum 180
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 128 columns.
Reduced LP has 229 rows, 1312 columns, and 22690 nonzeros.
Presolve time = 0.02 sec. (2.07 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2538.064492
self.times_called 16
get_num_nodes 7
values [-0.0, 1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x16', 'x18', 'x19', 'x22', 'x24', 'x27']
branc

Presolve time = 0.00 sec. (1.94 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2540.319968
self.times_called 28
get_num_nodes 13
values [-0.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x9', 'x12', 'x14', 'x16', 'x18', 'x19', 'x21', 'x22', 'x24', 'x25', 'x26', 'x27', 'x28']
branched [9, 12, 14, 16, 18, 19, 21, 22, 24, 25, 26, 27, 28]
     13    15     1352.2615     5     1171.0000     1422.8501       48   21.51%              x9 D     13     12     12
self.times_called 29
get_num_nodes 14
values [1.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x6', 'x9', 'x12', 'x14', 'x16', 'x18', 'x19', 'x21', 'x22', 'x24', 'x25', 'x26', 'x27', 'x28']
branched [6, 9, 12, 14, 16, 18, 19, 21, 22, 24, 25, 26, 27, 28]
MultiplyVar [0, 1, 2, 3, 4, 5, 7, 8]
NodeNum 22
ConstNum 180
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve

Presolve time = 0.00 sec. (1.87 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2977.919968
Iteration:    62   Scaled dual infeas =            24.898143
Iteration:   124   Scaled dual infeas =            23.647275
Iteration:   186   Scaled dual infeas =            23.035473
Iteration:   248   Scaled dual infeas =            17.445648
Iteration:   310   Scaled dual infeas =             8.976719
Iteration:   372   Scaled dual infeas =             1.709476
Iteration:   423   Dual objective     =             0.274997
Iteration:   484   Dual objective     =             0.041553
Iteration:   546   Dual objective     =             0.013145
Iteration:   608   Dual objective     =             0.007671
Iteration:   670   Dual objective     =             0.004406
Iteration:   732   Dual objective     =             0.003775
Iteration:   794   Dual objective     =             0.002412
Iteration:   856   Dual objective     =           

Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2158.933325
Iteration:    62   Scaled dual infeas =             1.945343
Iteration:   124   Scaled dual infeas =             1.945343
Iteration:   154   Dual objective     =             0.000000
Iteration:   215   Dual objective     =            -0.000000
Perturbation started.
Iteration:   255   Dual objective     =             0.000000
Iteration:   317   Dual objective     =            -0.000000
Removing perturbation.
self.times_called 52
get_num_nodes 26
values [-0.0, 1.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x9', 'x16', 'x19', 'x22', 'x24', 'x27']
branched [9, 16, 19, 22, 24, 27]
     26    21     1403.6034     3     1171.0000     1418.7155       73   21.15%              x9 D     26     25      6
self.times_called 53
get_num_nodes 27
values [1.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x8', 'x9', 'x16', 'x19', 'x22', 'x24', 'x27']
branched [8, 9, 16, 19, 22, 24, 27]
Mu

...

    327   139     1321.8000     1     1268.0000     1351.7966      764    6.61%              x8 U    327    326     10
*   328    71      integral     0     1317.0000     1351.7966      765    2.64%
    328    70     1317.0000     0     1317.0000     1351.7966      765    2.64%              x5 D    328    327     11
    329    69    infeasible           1317.0000     1350.7581      765    2.56%              x6 D    329     33     14
    330    68        cutoff           1317.0000     1348.7353      769    2.41%             x14 D    330     11     11
Elapsed time = 73.75 sec. (18.65 ticks, tree = 0.02 MB, solutions = 8)
    331    67    infeasible           1317.0000     1348.4643      769    2.39%             x23 D    331     37     18
    332    66        cutoff           1317.0000     1347.4388      771    2.31%             x26 D    332    155      4
self.times_called 514
get_num_nodes 333
values [1.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0]
branchedNames ['x

    344    59     1321.1902     3     1317.0000     1343.5227      792    2.01%              x9 U    344    244     11
    345    58        cutoff           1317.0000     1343.3457      797    2.00%             x18 D    345    121      5
    346    57        cutoff           1317.0000     1343.2959      803    2.00%             x26 D    346     74      5
    347    56        cutoff           1317.0000     1343.1060      808    1.98%              x8 D    347    138     11
    348    55        cutoff           1317.0000     1341.8878      812    1.89%             x26 D    348    169      6
    349    54        cutoff           1317.0000     1341.7064      814    1.88%              x9 U    349     12     12
    350    53        cutoff           1317.0000     1341.7064      818    1.88%             x28 D    350    145      7
Elapsed time = 75.22 sec. (19.21 ticks, tree = 0.02 MB, solutions = 8)
    351    52        cutoff           1317.0000     1341.6361      822    1.87%             x28 

    361    46     1328.0423     6     1317.0000     1337.0324      852    1.52%             x16 D    361    322      6
self.times_called 532
get_num_nodes 362
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x7', 'x8', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29']
branched [1, 2, 3, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29]
self.times_called 533
get_num_nodes 362
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x7', 'x8', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29']
branched [1, 2, 3, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23,

    378    34    infeasible           1317.0000     1330.6763      896    1.04%             x16 D    378    377      7
    379    33        cutoff           1317.0000     1330.4371      900    1.02%             x18 D    379    193      9
    380    32        cutoff           1317.0000     1329.7995      904    0.97%             x21 U    380     75      6
Elapsed time = 78.16 sec. (20.15 ticks, tree = 0.01 MB, solutions = 8)
self.times_called 542
get_num_nodes 381
values [1.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x2', 'x3', 'x9', 'x11', 'x12', 'x13', 'x14', 'x16', 'x17', 'x18', 'x19', 'x21', 'x22', 'x24', 'x26', 'x27', 'x28']
branched [2, 3, 9, 11, 12, 13, 14, 16, 17, 18, 19, 21, 22, 24, 26, 27, 28]
MultiplyVar [0, 1, 4, 5, 6, 7, 8, 10]
NodeNum 22
ConstNum 180
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 128 columns.
Reduced LP has 218 rows, 1312 columns, and 

self.times_called 550
get_num_nodes 396
values [1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x2', 'x3', 'x9', 'x10', 'x11', 'x13', 'x14', 'x15', 'x16', 'x17', 'x19', 'x21', 'x22', 'x26', 'x27', 'x28']
branched [2, 3, 9, 10, 11, 13, 14, 15, 16, 17, 19, 21, 22, 26, 27, 28]
MultiplyVar [0, 1, 4, 5, 6, 7, 8, 12]
NodeNum 22
ConstNum 180
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 128 columns.
Reduced LP has 219 rows, 1312 columns, and 20151 nonzeros.
Presolve time = 0.02 sec. (1.89 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =            12.423505
self.times_called 551
get_num_nodes 396
values [1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x2', 'x3', 'x9', 'x10', 'x11', 'x13', 'x14', 'x15', 'x16', 'x17', 'x19', 'x21', 'x22', 'x26', 'x27', 'x28']
branched [2,